In [ ]:
with open('./training_data/Roman-Urdu.txt', 'r', encoding='utf-8') as f:
    romanUrduLines = f.readlines()[:200]

with open('./training_data/Urdu.txt', 'r', encoding='utf-8') as f:
    urduLines = f.readlines()[:200]


    

In [ ]:

# Class to load data

class Dataset(torch.utils.data.Dataset):

    def __init__(self) -> None:
        self.source = open('./training_data/Roman-Urdu.txt', 'r', encoding='utf-8').read().split('\n')
        self.target = open('./training_data/Urdu.txt', 'r', encoding='utf-8').read().split('\n')


    def __getitem__(self, idx):
        source_sample = self.source[idx]
        target_sample = self.target[idx]

        return source_sample, target_sample

    def __len__(self):
        return len(self.source)
    

In [ ]:
yourDataset = Dataset()
dataloader = torch.utils.data.DataLoader(
        yourDataset,
        batch_size=8,
        num_workers=0,
        shuffle=True
)


In [ ]:
print(dataloader.)

In [119]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random


In [1]:
with open("./training_data/Roman-Urdu.txt", "r", encoding="utf-8") as f:
    roman_urdu_data = f.read()

with open("./training_data/Urdu.txt", "r", encoding="utf-8") as f:
    urdu_data = f.read()



In [2]:
print(urdu_data.splitlines())

['وکیل ', 'وکیل    ', 'وکیل     ایک ایسی شخصیت کو کہا جاتا ہے کہ جو دوسرے  اپنے صارف  کی جانب سے یا اسکی بابت گفتگو کرے   اس مضمون میں یہ گفتگو قانون سے متعلق تصور کی گئی ہے اور اس وجہ سے یہ مضمون صرف قانونی وکلا  کے بارے میں ذکر کرتا ہے', 'وکیل  قانون ', 'وکیل  قانون  ایک شخص جسے دوسرے شخص کی جگہ کام کرنے یا     کی نمائندگی کرنے کا اختیار حاصل ہوتا ہے  وکیل  سفر  ایک شخص جو تعطیلات اور سفر کا بندوبست کرتا ہے  خفیہ وکیل  ایک جاسوس ', 'وکیل بازار ضلع   لاطینی       ترکمانستان کا ایک ترکمانستان کے اضلاع جو صوبہ ماری میں واقع ہے', 'وکیل والا ریلوے اسٹیشن وکیل والا میں واقع ہے', 'وکیل والا ریلوے اسٹیشن پاکستان میں واقع ہے', 'وکیپیڈیا', 'وکیپیڈیا انگریزی پر یہ تصویر ميں نے ہی اپلوڈ کی تھی', 'وکیپیڈیا خود کسی بھی ترتیب کو نافذ نہیں کرتا', 'وکیپیڈیا لوگو کے لیے نستعلیق خطاطی', 'وکیپیڈیا میں اس سطر سے آپ تصاویر سادہ طریقے سے شامل کرسکتے ہیں ', 'وکیپیڈیا میں خوش آمدید', 'وکیپیڈیا میں کسی کے کو اس بات کی وجہ سے فوقیت حاصل نہیں کہ اس نے کس قدر وکیپیڈیا میں اضافہ کیا ہے', 'وکیپیڈیا پالیسی ', 'وکیپ

In [3]:
roman_urdu_lines = roman_urdu_data.splitlines()
urdu_lines = urdu_data.splitlines()

Output of urdu text has empty spaces which we need to remove

In [76]:
urdu_lines[2].split(' ')[:10]

['وکیل', '', '', '', '', 'ایک', 'ایسی', 'شخصیت', 'کو', 'کہا']

In [71]:
print(urdu_lines[2].replace('  ',' '))

وکیل   ایک ایسی شخصیت کو کہا جاتا ہے کہ جو دوسرے اپنے صارف کی جانب سے یا اسکی بابت گفتگو کرے  اس مضمون میں یہ گفتگو قانون سے متعلق تصور کی گئی ہے اور اس وجہ سے یہ مضمون صرف قانونی وکلا کے بارے میں ذکر کرتا ہے


In [77]:
[word for word in urdu_lines[2].split(' ') if word != ''][:10]

['وکیل', 'ایک', 'ایسی', 'شخصیت', 'کو', 'کہا', 'جاتا', 'ہے', 'کہ', 'جو']

In [82]:
for i, line in enumerate(urdu_lines):
    # Remove empty spaces for each line and also truncate to 10 words
    urdu_lines[i] = ' '.join([word for word in line.split(' ') if word != ''][:10])

We can see that now empty spaces are gone and the sentences are truncated to 10 words

In [83]:
urdu_lines[2].split(' ')
    

['وکیل', 'ایک', 'ایسی', 'شخصیت', 'کو', 'کہا', 'جاتا', 'ہے', 'کہ', 'جو']

Now we do the same for roman urdu

In [84]:
for i, line in enumerate(roman_urdu_lines):
    # Remove empty spaces for each line and also truncate to 10 words
    roman_urdu_lines[i] = ' '.join([word for word in line.split(' ') if word != ''][:10])

In [85]:
roman_urdu_lines[2].split(' ')

['wakeel',
 'aik',
 'aisi',
 'shakhsiyat',
 'ko',
 'kaha',
 'jata',
 'hai',
 'ke',
 'jo']

In [4]:
print(roman_urdu_lines, urdu_lines[3].split())

['wakeel', 'wakeel', 'wakeel aik aisi shakhsiyat ko kaha jata hai ke jo dosray apne Sarif ki janib se ya uski baabat guftagu kere is mazmoon mein yeh guftagu qanoon se mutaliq tasawwur ki gayi hai aur is wajah se yeh mazmoon sirf qanooni wukla ke baray mein zikar karta hai', 'wakeel qanoon', 'wakeel qanoon 1 shakhs jisay dosray shakhs ki jagah kaam karne ya ki numaindagi karne ka ikhtiyar haasil hota hai wakeel safar 1 shakhs jo tatilat aur safar ka bandobast karta hai khufia wakeel aik jasoos', 'wakeel bazaar zila lateeni turkmenistan ka aik turkmenistan ke azlaa jo soobah maari mein waqay hai', 'wakeel wala railway station wakeel wala mein waqay hai', 'wakeel wala railway station Pakistan mein waqay hai', 'wikipedia', 'wikipedia angrezi par yeh tasweer mein ne hi aplod ki thi', 'wikipedia khud kisi bhi tarteeb ko nafiz nahi karta', 'wikipedia logo ke liye nastalik khtati', 'wikipedia mein is satar se aap tasaveer saada tareeqay se shaamil kar saktay hain', 'wikipedia mein khush aamde

In [86]:
def getUniqueWords(lines):

    vocab = []
    for i in lines:
        toks = i.split()

        for j in toks:
            if j not in vocab:
                vocab.append(j)

    return vocab

Get Vocab of roman urdu and urdu

In [87]:
import torchtext.vocab as vocab

vocab_roman = getUniqueWords(roman_urdu_lines)
vocab_urdu = getUniqueWords(urdu_lines)

Urdu Vocab only first 35 words

In [89]:
print(len(vocab_urdu))
vocab_urdu[:35]

214


['وکیل',
 'ایک',
 'ایسی',
 'شخصیت',
 'کو',
 'کہا',
 'جاتا',
 'ہے',
 'کہ',
 'جو',
 'قانون',
 'شخص',
 'جسے',
 'دوسرے',
 'کی',
 'جگہ',
 'کام',
 'بازار',
 'ضلع',
 'لاطینی',
 'ترکمانستان',
 'کا',
 'کے',
 'اضلاع',
 'والا',
 'ریلوے',
 'اسٹیشن',
 'میں',
 'واقع',
 'پاکستان',
 'وکیپیڈیا',
 'انگریزی',
 'پر',
 'یہ',
 'تصویر']

In [90]:
print(len(vocab_roman))
vocab_roman[:35]

217


['wakeel',
 'aik',
 'aisi',
 'shakhsiyat',
 'ko',
 'kaha',
 'jata',
 'hai',
 'ke',
 'jo',
 'qanoon',
 '1',
 'shakhs',
 'jisay',
 'dosray',
 'ki',
 'jagah',
 'kaam',
 'bazaar',
 'zila',
 'lateeni',
 'turkmenistan',
 'ka',
 'azlaa',
 'wala',
 'railway',
 'station',
 'mein',
 'waqay',
 'Pakistan',
 'wikipedia',
 'angrezi',
 'par',
 'yeh',
 'tasweer']

In [91]:
f_zip = list(zip(vocab_roman, vocab_urdu))

In [92]:
print(f_zip)

[('wakeel', 'وکیل'), ('aik', 'ایک'), ('aisi', 'ایسی'), ('shakhsiyat', 'شخصیت'), ('ko', 'کو'), ('kaha', 'کہا'), ('jata', 'جاتا'), ('hai', 'ہے'), ('ke', 'کہ'), ('jo', 'جو'), ('qanoon', 'قانون'), ('1', 'شخص'), ('shakhs', 'جسے'), ('jisay', 'دوسرے'), ('dosray', 'کی'), ('ki', 'جگہ'), ('jagah', 'کام'), ('kaam', 'بازار'), ('bazaar', 'ضلع'), ('zila', 'لاطینی'), ('lateeni', 'ترکمانستان'), ('turkmenistan', 'کا'), ('ka', 'کے'), ('azlaa', 'اضلاع'), ('wala', 'والا'), ('railway', 'ریلوے'), ('station', 'اسٹیشن'), ('mein', 'میں'), ('waqay', 'واقع'), ('Pakistan', 'پاکستان'), ('wikipedia', 'وکیپیڈیا'), ('angrezi', 'انگریزی'), ('par', 'پر'), ('yeh', 'یہ'), ('tasweer', 'تصویر'), ('ne', 'ميں'), ('hi', 'نے'), ('aplod', 'ہی'), ('khud', 'اپلوڈ'), ('kisi', 'خود'), ('bhi', 'کسی'), ('tarteeb', 'بھی'), ('nafiz', 'ترتیب'), ('nahi', 'نافذ'), ('karta', 'نہیں'), ('logo', 'کرتا'), ('liye', 'لوگو'), ('nastalik', 'لیے'), ('khtati', 'نستعلیق'), ('is', 'خطاطی'), ('satar', 'اس'), ('se', 'سطر'), ('aap', 'سے'), ('tasaveer', '

Create a dictionary that maps each word to an index

In [129]:
roman_word_to_ix = {word: i for i, word in enumerate(vocab_roman)}
urdu_word_to_ix = {word: i for i, word in enumerate(vocab_urdu)}

REVERSE_roman_word_to_ix =  {v: k for k, v in roman_word_to_ix.items()}
REVERSE_urdu_word_to_ix =  {v: k for k, v in urdu_word_to_ix.items()}



print(roman_word_to_ix)
print(urdu_word_to_ix)


{'wakeel': 0, 'aik': 1, 'aisi': 2, 'shakhsiyat': 3, 'ko': 4, 'kaha': 5, 'jata': 6, 'hai': 7, 'ke': 8, 'jo': 9, 'qanoon': 10, '1': 11, 'shakhs': 12, 'jisay': 13, 'dosray': 14, 'ki': 15, 'jagah': 16, 'kaam': 17, 'bazaar': 18, 'zila': 19, 'lateeni': 20, 'turkmenistan': 21, 'ka': 22, 'azlaa': 23, 'wala': 24, 'railway': 25, 'station': 26, 'mein': 27, 'waqay': 28, 'Pakistan': 29, 'wikipedia': 30, 'angrezi': 31, 'par': 32, 'yeh': 33, 'tasweer': 34, 'ne': 35, 'hi': 36, 'aplod': 37, 'khud': 38, 'kisi': 39, 'bhi': 40, 'tarteeb': 41, 'nafiz': 42, 'nahi': 43, 'karta': 44, 'logo': 45, 'liye': 46, 'nastalik': 47, 'khtati': 48, 'is': 49, 'satar': 50, 'se': 51, 'aap': 52, 'tasaveer': 53, 'saada': 54, 'tareeqay': 55, 'khush': 56, 'aamdeed': 57, 'baat': 58, 'wajah': 59, 'policy': 60, 'sirf': 61, 'hazaar': 62, 'mazmoon': 63, 'hain': 64, 'jin': 65, 'asal': 66, 'har': 67, 'Sarif': 68, 'abna': 69, 'safha': 70, 'hota': 71, 'adad': 72, 'o': 73, 'shumaar': 74, 'sarfeen': 75, 'dayen': 76, 'janib': 77, 'talaash'

Convert To indeces

In [94]:
urdu_word_indices = torch.tensor([urdu_word_to_ix[word] for word in vocab_urdu], dtype=torch.long)
roman_word_indices = torch.tensor([roman_word_to_ix[word] for word in vocab_roman], dtype=torch.long)

In [95]:
print(urdu_word_indices)

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [26]:
print(roman_word_indices)


tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [96]:


embedding = nn.Embedding(num_embeddings=500, embedding_dim=128)


roman_embedded_words = embedding(roman_word_indices)
urdu_embedded_words = embedding(urdu_word_indices)

In [97]:
print(urdu_embedded_words)

tensor([[ 0.9712,  0.6001,  1.3942,  ..., -0.9436, -0.8932, -0.1931],
        [ 0.8179,  0.1684, -0.9584,  ..., -0.2299,  0.6909,  0.2241],
        [ 0.4038,  0.2324,  1.0719,  ...,  0.9289, -1.4149,  0.6941],
        ...,
        [ 0.4305,  0.4337,  0.1545,  ...,  0.3729, -0.5917, -0.0610],
        [-0.0384, -1.3454,  1.3223,  ...,  0.6655,  0.5884,  0.8072],
        [ 0.4210, -2.2935, -0.1947,  ..., -0.2525, -2.0656,  1.9400]],
       grad_fn=<EmbeddingBackward0>)


In [98]:
print(roman_embedded_words)

tensor([[ 0.9712,  0.6001,  1.3942,  ..., -0.9436, -0.8932, -0.1931],
        [ 0.8179,  0.1684, -0.9584,  ..., -0.2299,  0.6909,  0.2241],
        [ 0.4038,  0.2324,  1.0719,  ...,  0.9289, -1.4149,  0.6941],
        ...,
        [-0.3086,  0.5473,  0.6639,  ...,  0.5194, -0.3119, -0.7241],
        [-0.0522,  0.1437, -0.4276,  ..., -0.0088,  1.1825, -0.4129],
        [-2.1778, -0.1771, -0.0880,  ..., -0.9535, -4.3381, -0.9850]],
       grad_fn=<EmbeddingBackward0>)


In [43]:
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Now we write the encorder Rnn class

In [249]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
    # loop 10 times to implement 10 cells of rnn
        for i in range(10):
            output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)





In [250]:
class Attention(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super(Attention, self).__init__()
        self.dropout = torch.nn.Dropout(dropout)
        self.softmax = torch.nn.Softmax(dim=-1)
        self.scale = torch.sqrt(torch.tensor(d_model, dtype=torch.float32))

    def forward(self, q, k, v, mask=None):
        # q, k, v have shape (batch_size, seq_len, d_model)
        scores = torch.matmul(q, k.transpose(-1, -2)) / self.scale

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attention_weights = self.softmax(scores)
        attention_weights = self.dropout(attention_weights)

        # Compute weighted sum of values
        output = torch.matmul(attention_weights, v)

        return output, attention_weights

Below is our Decoder Class

In [263]:
class DecoderRNN(nn.Module):
    def __init__(self, output_size, hidden_size, num_layers = 10):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size*2, hidden_size)
        self.attention = Attention(hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encorder_output):
       
        output = self.embedding(input).view(1, 1, self.hidden_size)
        context, attention_weights = self.attention(hidden[-1].unsqueeze(0), encorder_output, encorder_output)
        
        output = torch.cat((output[:, :, :self.hidden_size], context), dim=2)
        output, hidden = self.gru(output, hidden)
        # output = torch.cat((output, context), dim=2)

        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [251]:
# class DecoderRNN(nn.Module):
#     def __init__(self, output_size, hidden_size, num_layers = 10):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         self.attention = Attention
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, input, hidden, encorder_output):
#         output = self.embedding(input).view(1, 1, -1)
#         output = F.relu(output)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden
    
#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

In [252]:
roman_urdu_lines[2]

'wakeel aik aisi shakhsiyat ko kaha jata hai ke jo'

Now we make pairs for training

In [253]:
pairs = list(zip(roman_urdu_lines, urdu_lines))

In [254]:
pairs[10]

('wikipedia khud kisi bhi tarteeb ko nafiz nahi karta',
 'وکیپیڈیا خود کسی بھی ترتیب کو نافذ نہیں کرتا')

In [255]:
def index_from_sentence(idx, sentence):
    return [idx[word] for word in sentence.split(' ')]

def index_to_word(idx, index):
    return []

def tensor_from_sentence(idx, sentence):
    indexes = index_from_sentence(idx, sentence)
    # Can add EOS Token Here if needed
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensors_from_pair(pair):
    input_tensor = tensor_from_sentence(roman_word_to_ix, pair[0])
    target_tensor = tensor_from_sentence(urdu_word_to_ix, pair[1])
    return (input_tensor, target_tensor)


We need to convert a sentence into vectors

In [256]:
print(index_from_sentence(roman_word_to_ix, roman_urdu_lines[5]))

print(tensors_from_pair([roman_urdu_lines[23], urdu_lines[23]]))

[0, 18, 19, 20, 21, 22, 1, 21, 8, 23]
(tensor([[91],
        [ 8],
        [92],
        [93],
        [22],
        [94],
        [40],
        [95],
        [96],
        [97]]), tensor([[90],
        [22],
        [91],
        [92],
        [21],
        [93],
        [41],
        [94],
        [95],
        [96]]))


Now we write out training function. Below code is implementing a training loop for a sequence-to-sequence

In [257]:
# Define teach forcing ratio and max length
teacher_forcing_ratio = 0.5
MAX_LENGTH = 10

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = decoder_input = input_tensor[-1].unsqueeze(0) 
    
    decoder_hidden = encoder_hidden

  

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if di + 1 == target_length:
                break


    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Helper Functions

In [258]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [259]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [260]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [261]:
PAD_token = 3

In [262]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensor_from_sentence(roman_word_to_ix, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = input_tensor[0].view(1, 1)
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            # decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if di == max_length - 1 or topi.item() == PAD_token:
                break
            else:
                decoded_words.append(REVERSE_urdu_word_to_ix[topi.item()])
            decoder_input = topi.squeeze().detach()
        
        return decoded_words, decoder_attentions[:di + 1]

In [264]:
hidden_size = 256
encoder1 = EncoderRNN(len(vocab_roman), hidden_size,).to(device)
attn_decoder1 = DecoderRNN( len(vocab_urdu), hidden_size).to(device)

trainIters(encoder1, attn_decoder1, 5000, print_every=500)

0m 43s (- 6m 33s) (500 10%) 3.6637
1m 29s (- 5m 58s) (1000 20%) 2.0055
2m 13s (- 5m 11s) (1500 30%) 0.7091
55m 5s (- 82m 38s) (2000 40%) 0.4111
56m 5s (- 56m 5s) (2500 50%) 0.3374
57m 12s (- 38m 8s) (3000 60%) 0.2350
58m 13s (- 24m 57s) (3500 70%) 0.2511
59m 9s (- 14m 47s) (4000 80%) 0.1860
60m 5s (- 6m 40s) (4500 90%) 0.1494
61m 4s (- 0m 0s) (5000 100%) 0.1635


In [265]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words,_ = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [268]:
evaluateRandomly(encoder1, attn_decoder1)

> woh mein bhartih jnta party mein shaamil hotay hue reyasti
= وہ میں بھارتیہ جنتا پارٹی میں شامل ہوتے ہوئے ریاستی
< وہ تا میں لاہور ہائی کورٹ بار ایسوسی ایشن

> woh taa mein Lahore high court baar association ke secretary
= وہ تا میں لاہور ہائی کورٹ بار ایسوسی ایشن کے
< وہ تا میں لاہور ہائی کورٹ بار ایسوسی ایشن

> woh mein misar ki bastii mhmodih ke aik ilm dost
= وہ میں مصر کی بستی محمودیہ کے ایک علم دوست
< وہ تا میں لاہور ہائی کورٹ بار ایسوسی ایشن

> wakeel wala railway station Pakistan mein waqay hai
= وکیل والا ریلوے اسٹیشن پاکستان میں واقع ہے
< وکیل قانون ایک شخص جسے دوسرے شخص کی جگہ

> wagar nah dil to usay rokna hi chahta hai
= وگر نہ دل تو اسے روکنا ہی چاہتا ہے
< وہ میں اقوام متحدہ گئے اور کشمیر پر اپنا

> woh mein wazeer taleem bhi reh chuke hain
= وہ میں وزیر تعلیم بھی رہ چکے ہیں
< وہ تا میں لاہور ہائی کورٹ بار ایسوسی ایشن

> woh ke aam intikhabaat mein Pasheen ke halqa se rukan
= وہ کے عام انتخابات میں پشین کے حلقہ سے رکن
< وہ تا میں لاہور ہائی کورٹ بار ایسوسی ایشن

> wakee

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device)
        out, hidden = self.rnn(x, h0)
        return out, hidden

In [ ]:
def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [37]:
# Split the data into training and validation sets
split_idx = int(0.8 * len(data))
train_data = data[:split_idx]
valid_data = data[split_idx:]

# Convert the sentences to PyTorch tensors
def to_tensor(sentence):
    
    token_ids = []
    for word in sentence:
        if word in vocab:
            token_ids.append(vocab[word])
        else:
            token_ids.append(vocab['<unk>'])
    return torch.tensor(token_ids, dtype=torch.long)

vocab = torchtext.vocab.build_vocab_from_iterator(
    [word for sentence in roman_urdu_sentences + urdu_sentences for word in sentence],
    specials=["<pad>", "<unk>"]
)

train_roman_urdu = [to_tensor(s) for s, _ in train_data if len(s) > 0]
train_urdu = [to_tensor(s) for _, s in train_data if len(s) > 0]
valid_roman_urdu = [to_tensor(s) for s, _ in valid_data if len(s) > 0]
valid_urdu = [to_tensor(s) for _, s in valid_data if len(s) > 0]

print(train_roman_urdu.size())



AttributeError: 'list' object has no attribute 'size'

ModuleNotFoundError: No module named 'mlflow'